In [1]:
import numpy as  np
import pandas as pd
from random import seed, random
import tensorflow as tf
from keras.models import Sequential

C:\Users\Pink\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data=[[[i+1+j] for i in range(3)] for j in range(3)]
data=np.array(data)

In [3]:
target=[(i+4) for i in range(3)]
target=np.array(target)

In [4]:
data,target

(array([[[1],
         [2],
         [3]],
 
        [[2],
         [3],
         [4]],
 
        [[3],
         [4],
         [5]]]), array([4, 5, 6]))

In [5]:
print('Shape of data -->',data.shape,'\nShape of target-->',target.shape)

Shape of data --> (3, 3, 1) 
Shape of target--> (3,)


In [6]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [7]:
def sigmoidPrime(output):
    return output*(1-output)

In [8]:
def RNNSeq(lr,epoch):
    
    
    hidden_dim=10
    input_dim = data.shape[1]
    output_dim = 1

    """
    Total Number of T is equal to length of data and we are adding one because we need to consider initial state
    which is all zero.
    int_s is a initial state also know as (h0)

    """ 

    np.random.seed(1)
    wxh = 2*np.random.random((hidden_dim,input_dim)) - 1
    whh = 2*np.random.random((hidden_dim,hidden_dim)) - 1
    why = 2*np.random.random((hidden_dim,output_dim)) - 1

    T = len(data)

    xs = np.zeros((T,hidden_dim))
    xs[-1]=np.zeros((hidden_dim))
    os = np.zeros((len(data),1))
    os= np.zeros(len(data))

    ### Starting of Gradient descent
    for ep in range(epoch):
        
        ### Run the code equals to length of data

        for i in range(len(data)):
            
            """ 
            ## Reshape is just broadcasting of Data
            Basically xs[i]=sigmoid(np.dot(wxh,data[i])+(np.dot(whh,xs[i-1])))

            """ 
            xs[i]=sigmoid(np.dot(wxh,data[i])+(np.dot(whh,xs[i-1])).reshape(hidden_dim,1)).reshape(hidden_dim)

            os[i]=sigmoid(np.dot(why.T,xs[i]))

            cost=np.sum((os[i]-target)**2)/2

            """
            Back-propogation through Time starts here

            Using Gradient Descent

            backward pass: compute gradients going backwards    
            initalize vectors for gradient values for each set of weights 

            """

            djdwhy=((os[i]-target[i])*sigmoidPrime(os[i])*xs[i]).reshape(hidden_dim,1)

            djdwxh=((os[i]-target[i])*sigmoidPrime(os[i])*why)*sigmoidPrime(xs[i]).reshape(hidden_dim,1)*data[i].T

            djdwhh=(((os[i]-target[i])*sigmoidPrime(os[i])*why)*sigmoidPrime(xs[i]).reshape(hidden_dim,1))*(xs[i-1].reshape(hidden_dim,1))


            ## SGD of why
            why=why-lr*djdwhy

            ## SGD of wxh
            wxh=wxh-lr*djdwxh

            ## SGD of whh
            whh=whh-lr*djdwhh
            
            
            # Assign new parameters to the model
            
            model={}
            
            model = {'Iteration':ep,'RMSE Loss':cost,'why': why,'whh': whh,'whx':wxh}
            
        if ep%400 ==0:
                
            print('Iteration =',ep,'\n','RMSE Loss',cost,'\n','why','\n',why,'\n','whh','\n',whh,'\n','wxh','\n',wxh)
            print('------------------------------------------------------------------------------------------')
                
    return model


In [9]:
RNNSeq(lr=.001,epoch=1200)

Iteration = 0 
 RMSE Loss 33.804311237163404 
 why 
 [[ 0.1717003 ]
 [ 0.93920588]
 [ 0.12219769]
 [-0.96007272]
 [ 0.60155384]
 [-0.53142823]
 [ 0.61502325]
 [-0.22130513]
 [ 0.729607  ]
 [ 0.49507955]] 
 whh 
 [[-0.80330491 -0.15778333  0.91578048  0.06633199  0.38375565 -0.36896732
   0.37300327  0.66925276 -0.96342203  0.50029005]
 [ 0.9777222   0.49633133 -0.43911199  0.57855868 -0.79354796 -0.10421292
   0.81719103 -0.41277168 -0.4244493  -0.73994283]
 [-0.96126583  0.35767132 -0.57674351 -0.46890642 -0.01685342 -0.89327465
   0.14823547 -0.70654259  0.17861133  0.39951698]
 [-0.79552897 -0.17208585  0.38860249 -0.17183929 -0.90029091  0.07159498
   0.32739146  0.0295804   0.88899168  0.17291225]
 [ 0.80681558 -0.72503884 -0.72143556  0.61479433 -0.20463458 -0.66927986
   0.85502891 -0.30445653  0.50163595  0.45200772]
 [ 0.76653685  0.24726908  0.50180953 -0.30227865 -0.46021955  0.7916971
  -0.14389296  0.92960476  0.32680766  0.2433161 ]
 [-0.77042991  0.89905667 -0.10009759  

{'Iteration': 1199, 'RMSE Loss': 25.1192950085921, 'why': array([[ 0.2180465 ],
        [ 0.94092558],
        [ 0.13652898],
        [-0.55224486],
        [ 0.83042646],
        [ 0.1914676 ],
        [ 1.01859944],
        [ 0.57546388],
        [ 1.49160188],
        [ 0.96157048]]), 'whh': array([[-0.80276786, -0.15724628,  0.91631753,  0.06686904,  0.3842927 ,
         -0.36843026,  0.37354033,  0.66978982, -0.96288497,  0.5008271 ],
        [ 0.97772416,  0.49633329, -0.43911004,  0.57856064, -0.79354601,
         -0.10421097,  0.81719299, -0.41276972, -0.42444734, -0.73994088],
        [-0.96123775,  0.3576994 , -0.57671544, -0.46887835, -0.01682535,
         -0.89324658,  0.14826354, -0.70651452,  0.1786394 ,  0.39954505],
        [-0.85284592, -0.2294028 ,  0.33128554, -0.22915624, -0.95760786,
          0.01427803,  0.27007451, -0.02773656,  0.83167473,  0.1155953 ],
        [ 0.84080135, -0.69105307, -0.68744978,  0.6487801 , -0.1706488 ,
         -0.63529408,  0.88901468, 